In [ ]:
from daft import serving

In [ ]:
BACKEND_CONFIGS = {
    "local": {"type": "docker"},
    "local-mp": {"type": "multiprocessing"},
    "aws_lambda": {"type": "aws_lambda", "execution_role_arn": "arn:aws:iam::941892620273:role/jay-daft-serving-testrole-OKTODELETE"},
}

In [ ]:
docker = serving.get_serving_backend(name="local", configs=BACKEND_CONFIGS)
mp = serving.get_serving_backend(name="local", configs=BACKEND_CONFIGS)
aws_lambda = serving.get_serving_backend(name="local", configs=BACKEND_CONFIGS)

In [ ]:
from daft.dataframe import DataFrame
from daft.serving.endpoint import HTTPEndpoint
from daft.expressions import col

source_df = DataFrame.from_pydict({"foo": [1, 2, 3]})
http_endpoint = HTTPEndpoint(source_df.schema() backend=docker)

df = DataFrame.from_endpoint(http_endpoint)
df = df.with_column("bar", col("foo") + 1)
df.write_endpoint(http_endpoint)

# HACK - until we implement runners, monkeypatch a mock function in
def endpoint_func(request: str) -> str:
    return request
http_endpoint._plan = endpoint_func

http_endpoint.deploy("my_df_endpoint")

In [ ]:
import requests
response = requests.get('http://localhost:56427?request=papa')
response.text